# Transformer ENCODER

In [1]:
import torch
import math
from torch import nn
import torch.nn.functional as F

In [2]:
d_model = 512 #dimensions of the word vec
num_heads = 8 #no. of heads in multihead attention i.e, no of paralliesd operations within the encoder
drop_prob = 0.1 #turn of neurons randomly
batch_size = 30 #faster, stable and reliable training i.e. parameters will be updated after 30 sentences in our case
max_sequence_length = 200
ffn_hidden = 2048 #512*4 : simply to learn additional information
num_layers = 5 #it's 6 in attention is all you need

In [3]:
#q,k,v = 30 x 8 x 200 x 64
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1] #64
    #in my understanding scaling is just to make it a normal distribution (mean=zero and variance=1), allows easier and stable training
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k) #30 x 8 x 200 x 200 ; 200 is the sequence length
    print(f"scaled.size() : {scaled.size()}")
    if mask is not None:
        print(f"-- ADDING MASK of shape {mask.size()} --")
        # Broadcasting add. So just the last N dimensions need to match(adding vectors of two different dimensions)
        # PyTorch is smart : 30 x 8 x 200 x 200
        scaled += mask
    #probability of how much should we focus
    attention = F.softmax(scaled, dim=-1)#30 x 8 x 200 x 200
    #have all info associated with context i.e. it knows which word to pay more attention
    values = torch.matmul(attention, v) # 30 x 8 x 200 x 64
    return values, attention



In [4]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model #512
        self.num_heads = num_heads #8
        self.head_dim = d_model // num_heads #64
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)# 512 x (512x3) : 3 for Q,K and V
        self.linear_layer = nn.Linear(d_model, d_model)#FFN 512 x 512

    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size() # 30 x 200 x 512
        print(f"x.size(): {x.size()}")
        qkv = self.qkv_layer(x) # 30 x 200 x 1536
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim) #30 x 200 x 8 x 192
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3) #30 x 8 x 200 x 192
        print(f"qkv.size(): {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1) # breaks entire permute thing tensor into 3 parts in accordance to the last dimension ; 30 x 8 x 200 x 64
        print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")
        values, attention = scaled_dot_product(q, k, v, mask)
        print(f"values.size(): {values.size()}, attention.size:{ attention.size()} ")
        values = values.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)
        print(f"values.size(): {values.size()}")
        out = self.linear_layer(values) # 30 x 200 x 512
        print(f"out.size(): {out.size()}")
        return out

In [5]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape #512
        self.eps=eps
        #gamma and beta are learnable parameyters
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) # 512
        self.beta =  nn.Parameter(torch.zeros(parameters_shape)) #512 ; mean


    def forward(self, inputs): #30 x 200 x 512
        dims = [-(i + 1) for i in range(len(self.parameters_shape))] # [-1]
        mean = inputs.mean(dim=dims, keepdim=True) # 30 x 200 x 1
        print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True) # 30 x 200 x 1
        std = (var + self.eps).sqrt() # 30 x 200 x 1
        print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std # 30 x 200 x 512
        print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta # 30 x 200 x 512
        print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
        print(f"out: {out.size()}")
        return out

In [6]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x): # 30 x 200 x 512
        x = self.linear1(x) # 30 x 200 x 2048
        print(f"x after first linear layer: {x.size()}")
        x = self.relu(x)  # 30 x 200 x 2048
        print(f"x after activation: {x.size()}")
        x = self.dropout(x)  # 30 x 200 x 2048
        print(f"x after dropout: {x.size()}")
        x = self.linear2(x)  # 30 x 200 x 512
        print(f"x after 2nd linear layer: {x.size()}")
        return x

In [7]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x):
        residual_x = x
        print("------- ATTENTION 1 ------")
        x = self.attention(x, mask=None)
        print("------- DROPOUT 1 ------")
        x = self.dropout1(x)
        print("------- ADD AND LAYER NORMALIZATION 1 ------")
        x = self.norm1(x + residual_x)
        residual_x = x
        print("------- ATTENTION 2 ------")
        x = self.ffn(x)
        print("------- DROPOUT 2 ------")
        x = self.dropout2(x)
        print("------- ADD AND LAYER NORMALIZATION 2 ------")
        x = self.norm2(x + residual_x)
        return x

In [8]:
class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        # 5 layers
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                     for _ in range(num_layers)])
    #overiding
    def forward(self, x):
      #pass through all 5 encoder layers
        x = self.layers(x)
        return x

In [10]:
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [ ]:
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # includes positional encoding
out = encoder(x)